In [1]:
import iofunctions as iof
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import numpy as np
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu
from scipy.special import stdtr

import seaborn as sns
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})


warnings.filterwarnings('ignore')
plt.style.use('ggplot')

mergedSpineDFMetaComplete = pd.read_excel('output/mergedSpineDFMetaComplete.xlsx','Sheet1')

mergedSpineDFMetaNormalApical = mergedSpineDFMetaComplete[(mergedSpineDFMetaComplete['oddType'] == "Normal")&(mergedSpineDFMetaComplete.dendriteType=="apical")&(mergedSpineDFMetaComplete.pyramidalLayer=="l5")].drop_duplicates()
mergedSpineDFMetaNormalBasal = mergedSpineDFMetaComplete[(mergedSpineDFMetaComplete['oddType'] == "Normal")&(mergedSpineDFMetaComplete.dendriteType=="basal")&(mergedSpineDFMetaComplete.pyramidalLayer=="l5")].drop_duplicates()

In [2]:
mergedSpineDFMetaNormalApical.tail()

,"Spine Length, um","Head Width, um",2D Spine Density 1/um,Straightness,Image Name,brainSlice,dendriteType,pyramidalLayer,Genotype,oddType
4413,2.64279,1.03308,0.576611,0.983935,MIN_apical_0004_w1TRANS Camera-1_Detailed.csv,2016-07-26_id9384_slide1_column2_slice7_100x_L...,apical,l5,WT,Normal
4414,1.42630,1.95882,0.576611,0.893766,MIN_apical_0004_w1TRANS Camera-1_Detailed.csv,2016-07-26_id9384_slide1_column2_slice7_100x_L...,apical,l5,WT,Normal
4415,2.00111,1.22988,0.576611,0.972127,MIN_apical_0004_w1TRANS Camera-1_Detailed.csv,2016-07-26_id9384_slide1_column2_slice7_100x_L...,apical,l5,WT,Normal
4416,2.06518,1.11854,0.576611,0.996846,MIN_apical_0004_w1TRANS Camera-1_Detailed.csv,2016-07-26_id9384_slide1_column2_slice7_100x_L...,apical,l5,WT,Normal
4417,1.02872,1.07155,0.576611,1.000000,MIN_apical_0004_w1TRANS Camera-1_Detailed.csv,2016-07-26_id9384_slide1_column2_slice7_100x_L...,apical,l5,WT,Normal
